# Chapter 4: Building Chatbots using LangChain

This notebook accompanies Chapter 4 of the LangChain guide, demonstrating practical implementations and setup instructions for building chatbots with LangChain.

Ensure you have Python 3.8+ installed before proceeding.


In [ ]:
import importlib
import subprocess
import sys

libraries = ["langchain", "langchain_community", "huggingface_hub", "langchain_openai"]

for library in libraries:
    try:
        # Try to import the library
        module = importlib.import_module(library)
        print(f"Library {library} version: {module.__version__}")
    except ImportError:
        # If library is not installed, attempt to install it
        print(f"Library {library} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", library])
        # After installing, import again and print the version
        module = importlib.import_module(library)
        # print(f"Library {library} version after installation: {module.__version__}")
    except AttributeError:
        # If library doesn't have __version__ attribute
        print(f"Library {library} does not have a __version__ attribute.")

Library langchain version: 0.3.23
Library langchain_community version: 0.3.21
Library huggingface_hub version: 0.30.1
Library langchain_openai does not have a __version__ attribute.


In [ ]:
import os
from langchain.llms import HuggingFaceHub
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from google.colab import userdata

**Connect to HuggingFace LLM**

In [ ]:
# Load LLaMA 4 model from Hugging Face Hub (make sure it's a chat-compatible LLaMA4 model)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')
llm = HuggingFaceHub(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",  # Adjust if you're using LLaMA 4 when it's available
    model_kwargs={
        "temperature": 0.7,
        "max_new_tokens": 256,
        "top_p": 0.9,
        "repetition_penalty": 1.1
    }
)

Azure OpenAI LLM

In [ ]:
import os
from langchain_openai import AzureOpenAI
from google.colab import userdata

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZ_OPENAI_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] =  "https://azopenai-demo.openai.azure.com/"

llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
embedding_model = AzureOpenAI(deployment_name="dp-text-embedding-ada-002", model_name="text-embedding-ada-002")

## Context Management Example

In [ ]:
# Initialize memory
memory = ConversationBufferMemory()

# Create ConversationChain with memory
chatbot = ConversationChain(llm=llm, memory=memory, verbose=True)

# Simulate a multi-turn conversation
print(chatbot.run("Hi, my name is Alice."))
print(chatbot.run("Can you help me with my account?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Alice.
AI:

> Finished chain.
 Hello Alice, it's nice to meet you. My name is AI, which stands for artificial intelligence. I am a computer program designed to simulate human conversation and assist with various tasks. What can I help you with today? 


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Alice.
AI:  Hello Alice, it's nice to meet you. My name i

## Tool Integration Example (Weather API)

In [ ]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

def get_weather(location):
    return f"The weather in {location} is sunny and 25°C."

weather_tool = Tool(
    name="Weather",
    func=get_weather,
    description="Provides weather updates for a given location."
)

# llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
agent = initialize_agent([weather_tool], llm, agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION)

print(agent.invoke("What's the weather in Paris?"))


{'input': "What's the weather in Paris?", 'output': 'The weather in Paris is sunny and 25°C.'}


## Multi-Turn Banking Bot

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI


# Step 2: Configure Memory for Context Maintenance
memory = ConversationBufferMemory()

# Step 3: Create the Conversation Chain
banking_bot = ConversationChain(llm=llm, memory=memory)

# Step 4: Define User Interactions
def banking_chat():
    print("Welcome to your banking assistant! Type 'exit' to end.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        response = banking_bot.run(user_input)
        print(f"Bot: {response}")

banking_chat()


Welcome to your banking assistant! Type 'exit' to end.
You: how to open bank account?
Bot:  To open a bank account, you will need to visit a bank branch or go online to the bank's website. You will need to provide personal information such as your name, address, Social Security number, and a form of identification. The bank may also require a minimum deposit to open the account. Once your account is open, you will receive a debit card and checks to use for your transactions. It's important to carefully read and understand the terms and conditions of the account before opening it. Do you have any specific questions about the process?
You: thanks
Bot:  You're welcome! Is there anything else I can assist you with?
You: exit


## Travel Assistant Query with Tool Integration

In [ ]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

# Define mock tools for flights and hotels
def find_flights(destination):
    return f"Flights to {destination}: AirFrance, Delta, and Lufthansa."

def find_hotels(location):
    return f"Hotels near {location}: Hotel A, Hotel B, and Hotel C."

flight_tool = Tool(
    name="Flight Finder",
    func=find_flights,
    description="Finds flights to a specified destination."
)

hotel_tool = Tool(
    name="Hotel Finder",
    func=find_hotels,
    description="Finds hotels near a specified location."
)

# Initialize LangChain agent
agent = initialize_agent([flight_tool, hotel_tool], llm, agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION)

# Handle the query
query = "Find me flights to Paris and a hotel near the Eiffel Tower."
response = agent.run(query)
print(response)


The best flight to take to Paris would be with AirFrance and the best hotel to stay at near the Eiffel Tower would be Hotel A.
